In [1]:
"""
Compact Dataset Generator for Energy Storage Optimization Fine-tuning

This generates training data with:
- Compressed JSON format (~400 tokens per output vs ~6,500)
- Simplified inputs (~2,000 tokens vs ~3,000)
- Complete sequences that fit in 4096 tokens
- Enhanced constraint prompting
"""

import json
import numpy as np
import pandas as pd
from datetime import datetime
from typing import List, Dict, Tuple
from pathlib import Path

# Import your existing framework
from agentic_energy.schemas import BatteryParams, DayInputs, SolveRequest, SolveResponse
from agentic_energy.data_loader import BatteryDataLoader
from agentic_energy.milp.milp_mcp_server import solve_daily_milp

Forecast Engine using device: cpu


In [2]:
class CompactDatasetGenerator:
    """Generate compact training data for fine-tuning"""
    
    def __init__(self, battery_params: BatteryParams):
        self.battery_params = battery_params
        self.instruction = self._create_instruction()
    
    def _create_instruction(self) -> str:
        """Create enhanced instruction with explicit constraints"""
        bp = self.battery_params
        
        return f"""You are an energy storage optimization expert. Determine the optimal 24-hour battery charge/discharge schedule to minimize costs.

BATTERY SPECIFICATIONS:
- Capacity: {bp.capacity_kwh:.2f} kWh
- Max charge/discharge power: {bp.cmax_kw:.2f} kW
- Charge efficiency: {bp.eta_c:.2f}
- Discharge efficiency: {bp.eta_d:.2f}
- Round-trip efficiency: {bp.eta_c * bp.eta_d:.4f}
- Initial SOC: {bp.soc_init:.2f}
- SOC range: [{bp.soc_min:.2f}, {bp.soc_max:.2f}]

PHYSICS & CONSTRAINTS:
SOC evolution equation:
  SOC[t+1] = SOC[t] + (charge[t] * {bp.eta_c} - discharge[t] / {bp.eta_d}) / {bp.capacity_kwh}

Hard constraints (ALL must be satisfied):
1. 0 ≤ charge[t] ≤ {bp.cmax_kw} kW for all hours
2. 0 ≤ discharge[t] ≤ {bp.dmax_kw} kW for all hours  
3. {bp.soc_min} ≤ SOC[t] ≤ {bp.soc_max} for all hours
4. charge[t] * discharge[t] = 0 (no simultaneous charge/discharge)
5. SOC[0] = {bp.soc_init}

OPTIMIZATION STRATEGY:
- Charge when price < 25th percentile
- Discharge when price > 75th percentile
- Maximize arbitrage spread (sell_price - buy_price) * efficiency
- Respect SOC limits throughout the day

OUTPUT FORMAT (JSON only, no explanations):
{{
  "status": "optimal",
  "cost": <total_cost_eur>,
  "charge": [<24 hourly values in kW>],
  "discharge": [<24 hourly values in kW>],
  "soc": [<25 values from hour 0 to 24>],
  "reasoning": "<brief 1-2 sentence strategy summary>"
}}"""
    
    def create_input(
        self,
        date: str,
        prices: np.ndarray,
        demand: np.ndarray,
        dataset_name: str = None
    ) -> str:
        """Create compact input component"""
        
        # Calculate essential statistics only
        price_stats = {
            "min": float(np.min(prices)),
            "max": float(np.max(prices)),
            "mean": float(np.mean(prices)),
            "p25": float(np.percentile(prices, 25)),
            "p75": float(np.percentile(prices, 75))
        }
        
        input_dict = {
            "date": date,
            "region": dataset_name.upper() if dataset_name else "UNKNOWN",
            "prices": [round(float(p), 2) for p in prices],
            "demand": [round(float(d), 1) for d in demand],
            "price_stats": {k: round(v, 2) for k, v in price_stats.items()}
        }
        
        return json.dumps(input_dict, separators=(',', ':'))  # Compact JSON
    
    def create_output(
        self,
        milp_solution: SolveResponse,
        prices: np.ndarray
    ) -> str:
        """Create compact output component"""
        
        # Extract solution
        charge = milp_solution.charge_kw or [0] * 24
        discharge = milp_solution.discharge_kw or [0] * 24
        soc = milp_solution.soc or [self.battery_params.soc_init] * 25
        cost = milp_solution.objective_cost or 0.0
        
        # Generate brief reasoning
        charge_hours = [h for h, c in enumerate(charge) if c > 0.01]
        discharge_hours = [h for h, d in enumerate(discharge) if d > 0.01]
        
        if charge_hours and discharge_hours:
            avg_charge_price = np.mean([prices[h] for h in charge_hours])
            avg_discharge_price = np.mean([prices[h] for h in discharge_hours])
            spread = avg_discharge_price - avg_charge_price
            
            reasoning = (
                f"Charge {len(charge_hours)}h at avg {avg_charge_price:.1f} EUR/MWh, "
                f"discharge {len(discharge_hours)}h at avg {avg_discharge_price:.1f} EUR/MWh. "
                f"Arbitrage spread: {spread:.1f} EUR/MWh."
            )
        else:
            reasoning = "No arbitrage opportunities - prices too flat."
        
        output_dict = {
            "status": milp_solution.status or "optimal",
            "cost": round(float(cost), 2),
            "charge": [round(float(c), 2) for c in charge],
            "discharge": [round(float(d), 2) for d in discharge],
            "soc": [round(float(s), 4) for s in soc],
            "reasoning": reasoning
        }
        
        return json.dumps(output_dict, separators=(',', ':'))  # Compact JSON
    
    def create_training_example(
        self,
        date: str,
        prices: np.ndarray,
        demand: np.ndarray,
        milp_solution: SolveResponse,
        dataset_name: str = None
    ) -> Dict[str, str]:
        """Create a complete training example"""
        
        return {
            "instruction": self.instruction,
            "input": self.create_input(date, prices, demand, dataset_name),
            "output": self.create_output(milp_solution, prices)
        }


def generate_compact_dataset(
    dataset_name: str,
    split: str,
    battery_params: BatteryParams,
    data_dir: str = "./agentic_energy/data",
    output_dir: str = "./datasets",
    max_days: int = None
) -> List[Dict[str, str]]:
    """Generate compact dataset for a specific region"""
    
    print(f"\n{'='*80}")
    print(f"GENERATING COMPACT DATASET: {dataset_name.upper()} ({split})")
    print(f"{'='*80}\n")
    
    # Configuration for each dataset
    dataset_config = {
        'italy': {
            'file': 'Italy_data.csv',
            'train_years': [2022],
            'test_years': [2023],
        },
        'germany': {
            'file': 'Germany_energy_Data.csv',
            'train_years': [2019, 2020, 2021, 2022],
            'test_years': [2023],
        },
        'caiso': {
            'file': 'CAISO_data.csv',
            'train_years': [2021, 2022],
            'test_years': [2023],
        }
    }
    
    if dataset_name not in dataset_config:
        raise ValueError(f"Unknown dataset: {dataset_name}")
    
    config = dataset_config[dataset_name]
    file_path = Path(data_dir) / config['file']
    
    # Load data
    print(f"Loading data from {file_path}...")
    df = pd.read_csv(file_path)
    df['timestamp'] = pd.to_datetime(df['timestamps'])
    df['year'] = df['timestamp'].dt.year
    df['date'] = df['timestamp'].dt.date

    # ✅ NEW: Convert to numeric (handles string values)
    df['prices'] = pd.to_numeric(df['prices'], errors='coerce')
    df['consumption'] = pd.to_numeric(df['consumption'], errors='coerce')

    # ✅ NEW: Remove rows with NaN values
    initial_len = len(df)
    df = df.dropna(subset=['prices', 'consumption'])
    if len(df) < initial_len:
        print(f"  Removed {initial_len - len(df)} rows with invalid data")

    # ✅ NEW: Germany-specific scaling (MW → kW)
    if dataset_name == 'germany':
        df['consumption'] = df['consumption'] / 1000
        print(f"  Applied Germany scaling: demand / 1000")
    
    # Filter by split
    years = config['train_years'] if split == 'train' else config['test_years']
    df_filtered = df[df['year'].isin(years)].copy()
    
    print(f"  Years: {years}")
    print(f"  Records: {len(df_filtered)}")
    
    # Get daily batches (only complete 24-hour days)
    daily_batches = []
    for date, group in df_filtered.groupby('date'):
        if len(group) == 24:
            prices = group['prices'].values
            demand = group['consumption'].values
            daily_batches.append((str(date), prices, demand))
    
    if max_days:
        daily_batches = daily_batches[:max_days]
    
    print(f"  Complete days: {len(daily_batches)}")
    
    # Generate training examples
    generator = CompactDatasetGenerator(battery_params)
    training_data = []
    
    print("\nGenerating examples...")
    for i, (date, prices, demand) in enumerate(daily_batches):
        if (i + 1) % 100 == 0:
            print(f"  Progress: {i+1}/{len(daily_batches)}")
        
        # Solve MILP
        day_inputs = DayInputs(
            prices_buy=prices.tolist(),
            demand_kw=demand.tolist()
        )
        solve_request = SolveRequest(
            battery=battery_params,
            day=day_inputs
        )
        
        try:
            # milp_solution = solve_daily_milp(solve_request)
            milp_solution = solve_daily_milp(
                batt=battery_params,
                day=day_inputs,
                solver="GUROBI",
                solver_opts={}
            )
            
            # Create training example
            example = generator.create_training_example(
                date=date,
                prices=prices,
                demand=demand,
                milp_solution=milp_solution,
                dataset_name=dataset_name
            )
            
            training_data.append(example)
            
            # Update SOC for next day (optional continuity)
            if milp_solution.soc and len(milp_solution.soc) > 0:
                battery_params.soc_init = milp_solution.soc[-1]
        
        except Exception as e:
            print(f"    Error on {date}: {e}")
            continue
    
    print(f"\n✓ Generated {len(training_data)} examples")
    
    # Save dataset
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True, parents=True)
    
    filename = f"{dataset_name}_{split}_compact.json"
    filepath = output_path / filename
    
    with open(filepath, 'w') as f:
        json.dump(training_data, f, indent=2)
    
    file_size_mb = filepath.stat().st_size / (1024 * 1024)
    
    print(f"\n✓ Dataset saved to: {filepath}")
    print(f"  Size: {file_size_mb:.2f} MB")
    print(f"  Examples: {len(training_data)}")
    
    return training_data

In [3]:
def analyze_token_lengths(dataset_path: str, tokenizer_name: str = "Qwen/Qwen2.5-1.5B-Instruct"):
    """Analyze token lengths of generated dataset"""
    from transformers import AutoTokenizer
    
    print(f"\n{'='*80}")
    print("TOKEN LENGTH ANALYSIS")
    print(f"{'='*80}\n")
    
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    with open(dataset_path, 'r') as f:
        data = json.load(f)
    
    # Alpaca template
    template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
    
    token_lengths = []
    for ex in data:
        text = template.format(ex["instruction"], ex["input"], ex["output"])
        tokens = tokenizer(text, truncation=False, add_special_tokens=True)
        token_lengths.append(len(tokens["input_ids"]))
    
    token_lengths = np.array(token_lengths)
    
    print(f"Dataset: {dataset_path}")
    print(f"Examples: {len(data)}\n")
    
    print("Token Length Statistics:")
    print(f"  Min:    {token_lengths.min():,}")
    print(f"  Max:    {token_lengths.max():,}")
    print(f"  Mean:   {token_lengths.mean():.0f}")
    print(f"  Median: {np.median(token_lengths):.0f}")
    print(f"  P95:    {np.percentile(token_lengths, 95):.0f}")
    print(f"  P99:    {np.percentile(token_lengths, 99):.0f}")
    
    for max_seq in [2048, 4096, 8192]:
        truncated = (token_lengths > max_seq).sum()
        pct = (truncated / len(token_lengths)) * 100
        print(f"\n  With max_seq_length={max_seq}:")
        print(f"    Truncated: {truncated}/{len(token_lengths)} ({pct:.1f}%)")
    
    # Show example
    print(f"\n{'='*80}")
    print("EXAMPLE (First Training Instance)")
    print(f"{'='*80}\n")
    
    example = data[0]
    print(f"Instruction length: {len(example['instruction'])} chars")
    print(f"Input length: {len(example['input'])} chars")
    print(f"Output length: {len(example['output'])} chars")
    print(f"Total tokens: {token_lengths[0]:,}")
    
    print("\n--- OUTPUT SAMPLE ---")
    print(example['output'][:500])
    if len(example['output']) > 500:
        print(f"... ({len(example['output']) - 500} more chars)")

In [4]:
# if __name__ == "__main__":
    # Battery configuration
battery_params = BatteryParams(
        capacity_kwh=49.44,
        cmax_kw=12.36,
        dmax_kw=12.36,
        eta_c=0.95,
        eta_d=0.95,
        soc_init=0.5,
        soc_min=0.0,
        soc_max=1.0,
        soc_target=0.5
    )
    
    # Generate datasets
datasets_to_generate = ['italy', 'germany', 'caiso']
    
all_train = []
all_test = []
    
for dataset_name in datasets_to_generate:
        # Reset battery SOC
        battery_params.soc_init = 0.5
        
        # Generate train split
        train_data = generate_compact_dataset(
            dataset_name=dataset_name,
            split='train',
            battery_params=battery_params,
            data_dir="./agentic_energy/data",
            output_dir="./datasets_compact",
            max_days=None  # Use all available days
        )
        all_train.extend(train_data)
        
        # Reset battery SOC
        battery_params.soc_init = 0.5
        
        # Generate test split
        test_data = generate_compact_dataset(
            dataset_name=dataset_name,
            split='test',
            battery_params=battery_params,
            data_dir="./agentic_energy/data",
            output_dir="./datasets_compact",
            max_days=None
        )
        all_test.extend(test_data)
    
    # Combine all datasets
print(f"\n{'='*80}")
print("COMBINING DATASETS")
print(f"{'='*80}\n")
    
combined_train_path = Path("./datasets_compact/combined_train_compact.json")
combined_test_path = Path("./datasets_compact/combined_test_compact.json")
    
with open(combined_train_path, 'w') as f:
        json.dump(all_train, f, indent=2)
    
with open(combined_test_path, 'w') as f:
        json.dump(all_test, f, indent=2)
    
print(f"✓ Combined train dataset: {combined_train_path}")
print(f"  Examples: {len(all_train)}")
print(f"  Size: {combined_train_path.stat().st_size / (1024**2):.2f} MB")
    
print(f"\n✓ Combined test dataset: {combined_test_path}")
print(f"  Examples: {len(all_test)}")
print(f"  Size: {combined_test_path.stat().st_size / (1024**2):.2f} MB")
    
    # Analyze token lengths
print(f"\n{'='*80}")
print("ANALYZING COMBINED TRAINING DATASET")
print(f"{'='*80}")
    
analyze_token_lengths(str(combined_train_path))




GENERATING COMPACT DATASET: ITALY (train)

Loading data from agentic_energy\data\Italy_data.csv...
  Years: [2022]
  Records: 8758
  Complete days: 364

Generating examples...
Set parameter Username


c:\Users\16467\OneDrive\Desktop\Columbia\Agentics\Another\Agentics_for_EnergyArbitrage_Battery\.venv\Lib\site-packages\cvxpy\expressions\expression.py:683: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)


[11/15/25 13:27:43] INFO     Set parameter Username                                              ]8;id=506290;file://c:\Users\16467\OneDrive\Desktop\Columbia\Agentics\Another\Agentics_for_EnergyArbitrage_Battery\.venv\Lib\site-packages\cvxpy\reductions\solvers\qp_solvers\gurobi_qpif.py\gurobi_qpif.py]8;;\:]8;id=219125;file://c:\Users\16467\OneDrive\Desktop\Columbia\Agentics\Another\Agentics_for_EnergyArbitrage_Battery\.venv\Lib\site-packages\cvxpy\reductions\solvers\qp_solvers\gurobi_qpif.py#160\160]8;;\

Academic license - for non-commercial use only - expires 2026-11-12


                    INFO     Academic license - for non-commercial use only - expires 2026-11-12 ]8;id=115474;file://c:\Users\16467\OneDrive\Desktop\Columbia\Agentics\Another\Agentics_for_EnergyArbitrage_Battery\.venv\Lib\site-packages\cvxpy\reductions\solvers\qp_solvers\gurobi_qpif.py\gurobi_qpif.py]8;;\:]8;id=486022;file://c:\Users\16467\OneDrive\Desktop\Columbia\Agentics\Another\Agentics_for_EnergyArbitrage_Battery\.venv\Lib\site-packages\cvxpy\reductions\solvers\qp_solvers\gurobi_qpif.py#160\160]8;;\

...

  Progress: 200/365


c:\Users\16467\OneDrive\Desktop\Columbia\Agentics\Another\Agentics_for_EnergyArbitrage_Battery\.venv\Lib\site-packages\cvxpy\expressions\expression.py:683: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1658 times so far.

  warnings.warn(msg, UserWarning)
c:\Users\16467\OneDrive\Desktop\Columbia\Agentics\Another\Agentics_for_EnergyArbitrage_Battery\.venv\Lib\site-packages\cvxpy\expressions\expression.py:683: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplicatio

  Progress: 300/365


c:\Users\16467\OneDrive\Desktop\Columbia\Agentics\Another\Agentics_for_EnergyArbitrage_Battery\.venv\Lib\site-packages\cvxpy\expressions\expression.py:683: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1758 times so far.

  warnings.warn(msg, UserWarning)
c:\Users\16467\OneDrive\Desktop\Columbia\Agentics\Another\Agentics_for_EnergyArbitrage_Battery\.venv\Lib\site-packages\cvxpy\expressions\expression.py:683: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplicatio


✓ Generated 365 examples

✓ Dataset saved to: datasets_compact\caiso_test_compact.json
  Size: 0.80 MB
  Examples: 365

COMBINING DATASETS

✓ Combined train dataset: datasets_compact\combined_train_compact.json
  Examples: 1093
  Size: 2.42 MB

✓ Combined test dataset: datasets_compact\combined_test_compact.json
  Examples: 729
  Size: 1.62 MB

ANALYZING COMBINED TRAINING DATASET

TOKEN LENGTH ANALYSIS

Dataset: datasets_compact\combined_train_compact.json
Examples: 1093

Token Length Statistics:
  Min:    1,168
  Max:    1,327
  Mean:   1202
  Median: 1183
  P95:    1273
  P99:    1294

  With max_seq_length=2048:
    Truncated: 0/1093 (0.0%)

  With max_seq_length=4096:
    Truncated: 0/1093 (0.0%)

  With max_seq_length=8192:
    Truncated: 0/1093 (0.0%)

EXAMPLE (First Training Instance)

Instruction length: 1207 chars
Input length: 414 chars
Output length: 544 chars
Total tokens: 1,225

--- OUTPUT SAMPLE ---
{"status":"optimal","cost":83261.27,"charge":[0.0,0.0,0.0,12.36,12.36,0.